# Collate the dataframes from step 1 into a single dataframe

In this step, we also:
- drop rows that are not Sumerian
- drop rows that don't have transliteration
- drop excess columns (only keep id, transliteration, period, and genre)
- drop duplicates
- standardize periods and genres

The result is saved to `outputs/2_tablets.csv`.

---

## Requirements

In [3]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path

tqdm.pandas()

PROJECT_ROOT = Path.cwd().parents[1]
OUTFILE = PROJECT_ROOT / "outputs" / "2_tablets.csv"

## Load files, join, filter out unwanted rows, standardize, save

In [6]:
def load_files_and_concat():
    # Load all files like "outputs/1_<corpus_name>.csv"
    # and concatenate them into a single dataframe
    # return the dataframe
    path = PROJECT_ROOT / "outputs"
    df = pd.concat([pd.read_csv(f) for f in path.glob("1_*.csv")])
    return df


df = load_files_and_concat()
print(f"Starting number of texts: {len(df)}")

Starting number of texts: 94410


/var/folders/gc/t2lltr8d7yb_hytn5gf12xbr0000gn/T/ipykernel_24168/2262418561.py:6: DtypeWarning: Columns (27,30,32,33,34,37,41,43,44,47,49,50,51,52,53,54,55,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in path.glob("1_*.csv")])


In [9]:
def filter_rows(df: pd.DataFrame) -> pd.DataFrame:
    # language is "Sumerian" or ""
    print("Dropping rows that are not Sumerian...")
    df = df[df["language"].isin(["Sumerian", ""])]
    df = df[~df["langs"].astype(str).str.contains("akk", na=False)]
    df = df[df["period"] != "Ebla"]
    df = df[df["period"] != "fake"]
    df = df[df["period"] != "Pre-Uruk V"]
    df = df[df["genre"] != "fake (modern)"]
    print(f"Updated number of texts: {len(df)}")
    print()

    # Print number of texts without transliteration
    print("Dropping rows without transliteration...")
    df = df[df["transliteration"] != ""]
    print(f"Updated number of texts: {len(df)}")
    print()

    # Drop duplicates based on id
    print("Drop duplicates...")
    df = df.drop_duplicates(subset="id")
    print(f"Updated number of texts: {len(df)}")
    print()
    print("✅ Done")

    return df


df = filter_rows(df)

Dropping rows that are not Sumerian...
Updated number of texts: 91296

Dropping rows without transliteration...
Updated number of texts: 91296

Drop duplicates...
Updated number of texts: 91296

✅ Done


In [10]:
def standardize(df: pd.DataFrame) -> pd.DataFrame:
    # Drop unnecessary columns
    df = df[["id", "transliteration", "period", "genre"]]

    # Standardize periods
    df.loc[df["period"].isin({"", "Uncertain"}), "period"] = "Unknown"

    # Standardize genres
    df.loc[df["genre"].isin({"", "uncertain"}), "genre"] = "Unknown"
    df.loc[df["genre"].isin({"Royal/Monumental", "Royal Inscription"}), "genre"] = (
        "Royal Inscription"
    )
    df.loc[df["genre"].isin({"Lexical; School", "Lexical"}), "genre"] = "Lexical"
    df.loc[df["genre"].isin({"Liturgy", "Ritual", "Hymn-Prayer"}), "genre"] = "Liturgy"
    df.loc[
        df["genre"].isin({"Mathematical", "Scientific", "Astronomical"}), "genre"
    ] = "Math/Science"
    return df


df = standardize(df)
print("✅ Done")

✅ Done


In [11]:
def print_stats(df: pd.DataFrame):
    print(df["period"].value_counts())
    print()
    print(df["genre"].value_counts())


print_stats(df)

period
Ur III                 79934
Old Akkadian            5288
Early Dynastic IIIb     3469
Old Babylonian           936
Early Dynastic IIIa      850
Lagash II                674
Early Dynastic I-II       88
Middle Babylonian          7
Neo-Babylonian             5
Neo-Assyrian               2
Unknown                    2
Name: count, dtype: int64

genre
Administrative       87406
Royal Inscription     1208
Letter                 804
Literary               785
Legal                  621
Lexical                 69
Liturgy                 63
Unknown                 60
Math/Science            13
Name: count, dtype: int64


In [12]:
def save(df: pd.DataFrame):
    print(f"Saving to csv at {OUTFILE}...")
    df.to_csv(OUTFILE, index=False)
    print()
    print("✅ Done")


save(df)

Saving to csv at /Users/cole/dev/sumerian/SumTablets/outputs/2_tablets.csv...

✅ Done
